In [283]:
import shapely.geometry as shg
import numpy as np
from shapely.geometry import CAP_STYLE, JOIN_STYLE
import copy
import pandas as pd
import math

In [284]:
class FunctionalZones:
    def __init__(self, value):
        self.value = value

    def __call__(self, s_site, kit = 1.6, liv_dens = 30, child = 0.065, child_dens = 35, pupil = 0.13, pupil_dens = 23, parking_perc = 0.8, mm = 0.42, mm_dens = 15, green = 6, places_insch = 550, places_inkind = 300):
        #1
        s_housing = s_site /(kit / (liv_dens/10000) * child * (child_dens / 10000) + kit / (liv_dens / 10000) * pupil * (pupil_dens / 10000) + kit / (liv_dens / 10000) * parking_perc * mm * (mm_dens / 10000) + kit / (liv_dens / 10000) * green / 10000 + 1)
        #2
        s_parking = s_housing * (kit / (liv_dens / 10000) * parking_perc * mm * (mm_dens / 10000))
        #3
        s_green = s_housing * (kit / (liv_dens / 10000) * green / 10000 )
        #4
        s_school = s_housing * (kit / (liv_dens / 10000) * pupil * (pupil_dens / 10000))
        #5
        s_kindgard = s_housing * (kit / (liv_dens/10000) * child * (child_dens / 10000))
        #6
        places_gou = s_housing*(kit/(liv_dens/10000)*pupil)
        #7
        places_dou = s_housing*(kit/(s_housing/10000)*child)
        #8
        people = s_housing*(kit/(s_housing/10000))
        #9
        def amount_gou():
            am_gou = round(places_gou / places_insch)
            if math.ceil((places_gou-(places_insch*am_gou)) / 275)<2:
                return am_gou
            else:
                return am_gou+1
        #10
        def amount_dou():
            am_dou = round(places_dou / places_inkind)
            if math.ceil(places_dou-(places_inkind*am_dou)) >= 250:
                return am_dou+1
            else:
                return am_dou


        return s_housing, s_parking, s_green, s_school, s_kindgard, places_gou, places_dou, people, amount_gou(), amount_dou()


In [285]:
s_h = FunctionalZones(5)

a,b,c,d,e,f,g,h,i,j = s_h(50, 1.6, 30, 0.065, 35, 0.13, 23, 0.8, 0.42, 15, 6, 1600, 300)
a

26.743688489516472

In [286]:
class Build:
    def __init__(self, **kwargs):
        self.__dict__|= kwargs
    def __call__(self):
        ans = copy.deepcopy(self.__dict__)
        ans.__delitem__('section')
        return np.asarray(list(ans.values()), dtype=float), np.asarray(ans.keys(), dtype=str)
    def __str__(self):
        return self.__dict__.__str__()


class Section:
    def __init__(self, depth=16.0, eak=0.75):
        self.section_depth = depth
        self.eak=eak


    def generate_section(self, cx, cy):
        CAP_STYLE.square
        JOIN_STYLE.mitre

        cell_rect = shg.box(minx= -cx / 2 , miny= -cy / 2, maxx= cx / 2 , maxy= cy / 2)
        w=list(cell_rect.boundary.coords)[:-1]

        ls=shg.LineString([*w])
        return ls.buffer(self.section_depth,  single_sided=True)

    def __call__(self, **kwargs):
        kwargs|={'section_depth':self.section_depth, 'eak':self.eak}
        ss = self.generate_section(kwargs['cx'],kwargs['cy'])
        nkwargs={}
        nkwargs.update(kwargs)
        nkwargs.update({'section': ss, 'area': ss.area*kwargs['floor']*self.eak, 'area_all':ss.area*kwargs['floor']})
        return Build(**nkwargs)




In [287]:
df = pd.read_csv('/Users/sofyadobycina/Downloads/aff_grid - aff_grid.csv')

for i in df:
    if df[i].dtype == object:
        df[i] = df[i].str.replace(',', '.').astype(float)

build =[]
for i,v in enumerate (df.Сx):
    cell = Section()
    build.append(cell(cx=v, cy=df.Сy[i], floor = df.floor[i]))

df['kit'] = [(v.area * 0.75) / (df.Slot[i]) for i, v in enumerate(build)]
df.Slot = df.Slot/10000
df

,floor,Сx,Сy,Ax,Ay,Bx,By,r,KZ,KPZ,Lx,Ly,Slot,kit
0,5,70,60.0,34.6,60.0,70,51.2,90,22,10867.05,104.6,111.2,1.163152,0.611270
1,5,70,60.0,34.6,60.0,70,28.3,90,27,13577.67,104.6,88.3,0.923618,0.769799
2,5,70,60.0,27.8,60.0,70,26.5,90,30,14949.07,97.8,86.5,0.845970,0.840455
3,5,70,75.0,32.6,75.0,70,61.0,90,22,10778.57,102.6,136.0,1.395360,0.606295
4,5,70,75.0,32.6,75.0,70,34.6,90,27,13374.84,102.6,109.6,1.124496,0.752337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,16,90,92.0,77.8,92.0,90,60.2,90,15,24257.89,167.8,152.2,2.553916,1.364493
68,16,90,92.0,42.6,92.0,90,41.6,90,22,34970.90,132.6,133.6,1.771536,1.967107
69,24,100,80.0,115.0,80.0,100,117.9,90,9,17143.78,215.0,197.9,4.254850,1.157456
70,24,100,80.0,77.8,80.0,100,81.8,90,13,25361.48,177.8,161.8,2.876804,1.711900


In [288]:
width = np.asarray([123.93118099999998, 129.39932800000003, 196.405932, 429.911573])
height = np.asarray([404.06521, 129.87100999999996, 214.29481500000009, 230.68462999999997])
mask = np.array([[False, False, False, False], [False, False, False, False], [False, False, True, True], [True, False, True, True]])
area = np.outer(height, width)/10000
area[mask] = 0

area

array([[ 5.00762787,  5.22857666,  7.93608042, 17.371231  ],
       [ 1.60950676,  1.68052214,  2.55074368,  5.58330502],
       [ 2.65578095,  2.77296051,  0.        ,  0.        ],
       [ 0.        ,  2.98504361,  0.        ,  0.        ]])

In [289]:
def types_calc (w, h, fl, site_area, sec_data, housing_area, kit=1.6):
    site_perc = site_area * 100 / np.sum(site_area)
    house_perc_area = housing_area * site_perc / 100
    Val = sec_data[(sec_data['floor'] == 12) & (kit-0.5>=sec_data['kit'] >= kit+0.5)]

    return Val


In [290]:
types_calc(width, height, [9,12], area, df, a)

,floor,Сx,Сy,Ax,Ay,Bx,By,r,KZ,KPZ,Lx,Ly,Slot,kit
35,12,75,65.0,40.6,65.0,75,33.6,90,24,29152.18,115.6,98.6,1.139816,1.639212
38,12,75,81.5,40.8,81.5,75,33.8,90,25,29648.63,115.8,115.3,1.335174,1.666300
41,12,75,98.0,40.8,98.0,75,33.8,90,25,30091.87,115.8,131.8,1.526244,1.691211
44,12,75,114.5,40.8,114.5,75,33.8,90,25,30436.54,115.8,148.3,1.717314,1.710578
47,12,75,130.0,40.6,130.0,75,33.8,90,26,30697.07,115.6,163.8,1.893528,1.728203
50,12,75,92.0,40.6,92.0,75,34.8,90,25,29733.91,115.6,126.8,1.465808,1.672525
